In [1]:
from find_drives import *

In [42]:
import plotly.express as px

In [41]:
import plotly.graph_objects as go  # or plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json
from textwrap import dedent as d

In [70]:
import pandas as pd

In [71]:
d = {
    'a': [1, 2, 3],
    'b': [4, 5, 6]
}

In [72]:
pd.DataFrame(d)

,a,b
0,1,4
1,2,5
2,3,6


Exception in thread Thread-1802 (<lambda>):
Traceback (most recent call last):
  File "/Users/danielpilsbury/.pyenv/versions/3.11.2/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Users/danielpilsbury/.pyenv/versions/3.11.2/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/danielpilsbury/.pyenv/versions/3.11.2/lib/python3.11/site-packages/dash/dash.py", line 1702, in <lambda>
    target=lambda: _watch.watch(
                   ^^^^^^^^^^^^^
  File "/Users/danielpilsbury/.pyenv/versions/3.11.2/lib/python3.11/site-packages/dash/_watch.py", line 35, in watch
    walk()
  File "/Users/danielpilsbury/.pyenv/versions/3.11.2/lib/python3.11/site-packages/dash/_watch.py", line 20, in walk
    info = os.stat(path)
           ^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/Users/danielpilsbury/.pyenv/versions/3.11.2/lib/python3.11/site-packages/dash/dcc/__pycache__/express.cpytho

In [3]:
args = {
    'season': 2018,
    'player_name': 'Stephen Curry',
    'start_loc': (-40, -22),
    'num_results': 5
}

In [4]:
graph_data = get_graph_data(**args)

In [45]:
def get_basketball_court_fig():
    fig = px.scatter()
    draw_basketball_court(fig)
    return fig

In [69]:
app = dash.Dash()

# fig = get_drive_animation(graph_data)
fig = get_basketball_court_fig()

# global state
graph_data = None
play_index = 0
current_traces = (None, None)

fig.update_layout(dragmode='lasso')
fig.update_layout(clickmode='event+select')



dropdown_options = [{"label": str(i), "value": i} for i in range(args['num_results'])]

app.layout = html.Div(
    [
        dcc.Graph(id="graph-court", figure=fig),
        html.Button('Reset', id='button-reset'),
        dcc.Dropdown(id="dropdown", options=dropdown_options, value=0),
         html.Div(
            className="row",
            children=[
                html.Div(
                    [
                        dcc.Markdown(d("""Selected Data using figure references""")),
                        html.Pre(id="selected_data"),
                    ],
                    className="three columns",
                ),
            ]
        ),
    ]
)

@app.callback(
    Output('graph-court', 'figure'),
    Input('button-reset', 'n_clicks'))
def reset_graph(n_clicks):
    global fig
    fig = get_basketball_court_fig()
    fig.update_layout(dragmode='lasso')
    fig.update_layout(clickmode='event+select')
    return fig
    
@app.callback(
    Output('graph-court', 'figure', allow_duplicate=True),
    Output('selected_data', 'children'),
    Input('graph-court', 'selectedData'),
    prevent_initial_call=True)
def display_selected_data(selectedData):
    global fig
    if selectedData and 'lassoPoints' in selectedData:
        lasso_points = selectedData['lassoPoints']
        x = lasso_points['x']
        y = lasso_points['y']
        neg_x = [-x for x in x]
        neg_y = [-y for y in y]


        args['start_loc'] = (x[0], y[0])
        global graph_data
        graph_data = get_graph_data(**args)
        fig = get_drive_animation(graph_data)
            
        # add a line trace for selected points
        global current_traces
        trace_1 = go.Scatter(x=x, y=y, mode='lines', line=dict(color='gray', dash='dash'), showlegend=False)
        trace_2 = go.Scatter(x=neg_x, y=neg_y, mode='lines', line=dict(color='gray', dash='dash'), showlegend=False)
        current_traces = (trace_1, trace_2)
        for trace in current_traces:
            fig.add_trace(trace)

        return fig, json.dumps(selectedData, indent=2)
    
    return fig, None

@app.callback(
    Output('graph-court', 'figure', allow_duplicate=True),
    Input('dropdown', 'value'),
    prevent_initial_call=True)
def update_graph(value):
    if isinstance(value, int):
        global fig
        global play_index
        play_index = value
        fig = get_drive_animation(graph_data, play_index)
        for trace in current_traces:
            fig.add_trace(trace)
        return fig
    assert False


app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


In [46]:
get_basketball_court_fig().show()

### Old

In [ ]:
# import plotly.graph_objects as go  # or plotly.express as px


# import dash
# import dash_core_components as dcc
# import dash_html_components as html

# app = dash.Dash()

# styles = {"pre": {"border": "thin lightgrey solid", "overflowX": "scroll"}}

# fig = get_drive_animation(graph_data)

# GS = 100
# # fig.add_traces(
# #     px.scatter(
# #         x=np.repeat(np.linspace(-50, 50, GS), GS),
# #         y=np.tile(np.linspace(-25, 25, GS), GS),
# #     )
# #     .update_traces(marker_color="rgba(0,0,0,0)")
# #     .data
# # )

# # fig.update_layout(dragmode='drawopenpath')
# fig.update_layout(clickmode='event+select')



# options = [{"label": str(i), "value": i} for i in graph_data]

# clicked = []

# app.layout = html.Div(
#     [
#         dcc.Dropdown(id="dropdown", options=options, value=0),
#         dcc.Graph(id="graph-court", figure=fig),
#         html.Div(
#             className="row",
#             children=[
#                 html.Div(
#                     [
#                         dcc.Markdown(d("""Hoverdata using figure references""")),
#                         html.Pre(id="hoverdata2", style=styles["pre"]),
#                     ],
#                     className="three columns",
#                 ),
#                 html.Div(
#                     [
#                         dcc.Markdown(
#                             d("""Full hoverdata""")
#                         ),
#                         html.Pre(id="hoverdata1", style=styles["pre"]),
#                     ],
#                     className="three columns",
#                 ),
#             ],
#         ),
#     ]
# )


# store = []


# # @app.callback(
# #     Output("graph-court", "figure"),
# #     Output("hoverdata1", "children"),
# #     # Output("hoverdata2", "children"),
# #     [Input("graph-court", "clickData")],
# #     prevent_initial_call=True,
# # )
# # def display_hover_data(hoverData):
# #     if hoverData is not None:
# #         traceref = hoverData["points"][0]["curveNumber"]
# #         pointref = hoverData["points"][0]["pointNumber"]
# #         store.append(
# #             [
# #                 fig.data[traceref]["name"],
# #                 fig.data[traceref]["x"][pointref],
# #                 fig.data[traceref]["y"][pointref],
# #             ]
# #         )
# #         fig.update_layout(title="Last label was " + fig.data[traceref]["name"])
# #         return fig, json.dumps(hoverData, indent=2)
# #     else:
# #         return fig, "None selected"
    
# @app.callback(
#     Output('graph-court', 'figure'),
#     Input('graph-court', 'selectedData'))
# def display_selected_data(selectedData):
#     if selectedData:
#         lasso_points = selectedData['lassoPoints']
#         x = lasso_points['x']
#         y = lasso_points['y']
#         # add a line trace for selected points
#         fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='gray', dash='dash'), showlegend=False))

#         return fig

# @app.callback(
#     Output('hoverdata2', 'children'),
#     Input('graph-court', 'clickAnnotationData'))
# def display_selected_data(selectedData):
#     return json.dumps(selectedData, indent=2)


# app.run_server(debug=True, use_reloader=False)